In [2]:
import numpy as np
import pandas as pd
import csv
pd.set_option('display.max_rows',500)

## Deforestation ##
##### (and maybe carbon capture if I understood which data we're using) 

In [3]:
DEFRST = pd.read_csv('FAOSTAT_data_en_3-10-2024 (1).csv')
#loading in data from 'https://www.fao.org/faostat/en/#data/RL'; in 'Elements', select 'carbon stock in living biomass', and in items, select all 'Forestland' 
#these filter selections can be completely wrong so have good documentation as you go pls

FileNotFoundError: [Errno 2] No such file or directory: 'FAOSTAT_data_en_3-10-2024 (1).csv'

In [3]:
defrst=DEFRST.drop(columns=['Flag Description', 'Flag','Year Code','Domain Code','Element Code','Item Code','Note', 'Unit'])

### edit units ###

In [1]:
defrst['Unit']= "Million metric tonnes"
defrst.to_csv("cleanedDEFRST.csv")

NameError: name 'defrst' is not defined

In [5]:
defrst.head()

,Domain,Area Code (M49),Area,Element,Item,Year,Value,Unit
0,Land Use,4,Afghanistan,Area,Forest land,1990,1208.44,million metric tonnes
1,Land Use,4,Afghanistan,Area,Forest land,1991,1208.44,million metric tonnes
2,Land Use,4,Afghanistan,Area,Forest land,1992,1208.44,million metric tonnes
3,Land Use,4,Afghanistan,Area,Forest land,1993,1208.44,million metric tonnes
4,Land Use,4,Afghanistan,Area,Forest land,1994,1208.44,million metric tonnes


In [6]:
b=defrst.loc[defrst['Value']==0.0] #most countries with no carbon stock value are edge case places
b.groupby("Area")["Value"].unique()
#these are the countries with no value for forest land

Area
Afghanistan                                     [0.0]
American Samoa                                  [0.0]
Andorra                                         [0.0]
Aruba                                           [0.0]
Bahamas                                         [0.0]
Bahrain                                         [0.0]
Barbados                                        [0.0]
Bonaire, Sint Eustatius and Saba                [0.0]
Botswana                                        [0.0]
Cayman Islands                                  [0.0]
Curaçao                                         [0.0]
Djibouti                                        [0.0]
Egypt                                           [0.0]
Equatorial Guinea                               [0.0]
Falkland Islands (Malvinas)                     [0.0]
Faroe Islands                                   [0.0]
Gibraltar                                       [0.0]
Greenland                                       [0.0]
Guam                   

## Carbon Capture: computed through Carbon Capture & Deforestation scores

In [7]:
CCap = pd.read_csv('FAOSTAT_data_en_3-10-2024 (2).csv')

In [8]:
ccap=CCap.drop(columns=['Flag Description', 'Domain', 'Item','Flag', 'Area Code (M49)','Year Code','Domain Code','Element Code','Item Code','Note', 'Unit'])
ccap

,Area,Element,Year,Value
0,Algeria,Carbon stock in living biomass,1990,31.1396
1,Algeria,Carbon stock in living biomass,1991,30.9831
2,Algeria,Carbon stock in living biomass,1992,30.8266
3,Algeria,Carbon stock in living biomass,1993,30.6701
4,Algeria,Carbon stock in living biomass,1994,30.5136
...,...,...,...,...
6478,Zimbabwe,Carbon stock in living biomass,2017,1004.8564
6479,Zimbabwe,Carbon stock in living biomass,2018,1002.2235
6480,Zimbabwe,Carbon stock in living biomass,2019,999.5906
6481,Zimbabwe,Carbon stock in living biomass,2020,996.7833


In [9]:
a=CCap.loc[CCap['Value']==0.0] #most countries with no carbon stock value are edge case places
a.groupby("Area")["Value"].unique()

Area
Falkland Islands (Malvinas)    [0.0]
Gibraltar                      [0.0]
Holy See                       [0.0]
Malta                          [0.0]
Monaco                         [0.0]
Nauru                          [0.0]
Qatar                          [0.0]
Tokelau                        [0.0]
Name: Value, dtype: object

#### Now, the problem is that to compute the final carbon capture score, it is the “percentage change in the ratio of carbon stock in living biomass over forest land from a 1990s average.” First, we compute an approximate 1990s average for forest land by averaging the 1990 and thereafter collected values (ie, iterate the averaging score across all years since 1980 for ALL countries). Then,  we compute the ratios for 1990s and 2018. Finally, we compute the percentage change in the ratios.

In [10]:
ccap_1990=ccap.loc[ccap['Year']==1990]
ccap_1990.head()

,Area,Element,Year,Value
0,Algeria,Carbon stock in living biomass,1990,31.1396
32,American Samoa,Carbon stock in living biomass,1990,1.1540
64,Andorra,Carbon stock in living biomass,1990,1.3898
96,Angola,Carbon stock in living biomass,1990,1369.6608
128,Anguilla,Carbon stock in living biomass,1990,0.6954


In [11]:
ccap_1990[['Area','Value']].rename(columns={'Value':'Value in 1990'})

,Area,Value in 1990
0,Algeria,31.1396
32,American Samoa,1.1540
64,Andorra,1.3898
96,Angola,1369.6608
128,Anguilla,0.6954
160,Antigua and Barbuda,1.2782
192,Argentina,3664.3844
254,Australia,8451.9833
286,Austria,334.7509
318,Bahamas,13.3532


In [12]:
merged_ccap=pd.merge(ccap, ccap_1990, on='Area')
clean_ccap=merged_ccap.rename(columns={'Value_x':'Value', 'Value_y': 'Value in 1990', 'Year_y':'Base year (1990)', 'Element_x':'Element', 'Year_x':'Year'}).drop(columns=['Element_y'])

In [13]:
clean_ccap['Average Natural Carbon Stock']=clean_ccap[['Value', 'Value in 1990']].mean(axis=1)
clean_ccap

,Area,Element,Year,Value,Base year (1990),Value in 1990,Average Natural Carbon Stock
0,Algeria,Carbon stock in living biomass,1990,31.1396,1990,31.1396,31.13960
1,Algeria,Carbon stock in living biomass,1991,30.9831,1990,31.1396,31.06135
2,Algeria,Carbon stock in living biomass,1992,30.8266,1990,31.1396,30.98310
3,Algeria,Carbon stock in living biomass,1993,30.6701,1990,31.1396,30.90485
4,Algeria,Carbon stock in living biomass,1994,30.5136,1990,31.1396,30.82660
...,...,...,...,...,...,...,...
5626,Zimbabwe,Carbon stock in living biomass,2017,1004.8564,1990,1189.2814,1097.06890
5627,Zimbabwe,Carbon stock in living biomass,2018,1002.2235,1990,1189.2814,1095.75245
5628,Zimbabwe,Carbon stock in living biomass,2019,999.5906,1990,1189.2814,1094.43600
5629,Zimbabwe,Carbon stock in living biomass,2020,996.7833,1990,1189.2814,1093.03235


In [14]:
defrst.head()

,Domain,Area Code (M49),Area,Element,Item,Year,Value,Unit
0,Land Use,4,Afghanistan,Area,Forest land,1990,1208.44,million metric tonnes
1,Land Use,4,Afghanistan,Area,Forest land,1991,1208.44,million metric tonnes
2,Land Use,4,Afghanistan,Area,Forest land,1992,1208.44,million metric tonnes
3,Land Use,4,Afghanistan,Area,Forest land,1993,1208.44,million metric tonnes
4,Land Use,4,Afghanistan,Area,Forest land,1994,1208.44,million metric tonnes


In [15]:
clean_defrst=defrst[['Area', 'Item', 'Year','Value']]
#pd.merge(ccap, ccap_1990, on='Area')

In [16]:
merged_cc=pd.merge(clean_defrst, clean_ccap, on='Area').drop(columns=['Year_y']).rename(columns={'Value_x':'Forest Land Value', 'Year_x': 'Year', 'Value_y': 'Carbon Value', 'Value in 1990':'Carbon Value in 1990'})

In [17]:
merged_cc['Carbon Capture']=merged_cc['Average Natural Carbon Stock']/merged_cc['Forest Land Value']
merged_cc['Unit']='million metric tonnes'
merged_cc['Description']='Natural carbon capture is computed as the ratio between the average change in natural carbon capture since 1990 to the current forest land'
clean_carbon_capture=merged_cc
clean_carbon_capture

,Area,Item,Year,Forest Land Value,Element,Carbon Value,Base year (1990),Carbon Value in 1990,Average Natural Carbon Stock,Carbon Capture,Unit,Description
0,Algeria,Forest land,1990,1667.0,Carbon stock in living biomass,31.1396,1990,31.1396,31.13960,0.018680,million metric tonnes,Natural carbon capture is computed as the rati...
1,Algeria,Forest land,1990,1667.0,Carbon stock in living biomass,30.9831,1990,31.1396,31.06135,0.018633,million metric tonnes,Natural carbon capture is computed as the rati...
2,Algeria,Forest land,1990,1667.0,Carbon stock in living biomass,30.8266,1990,31.1396,30.98310,0.018586,million metric tonnes,Natural carbon capture is computed as the rati...
3,Algeria,Forest land,1990,1667.0,Carbon stock in living biomass,30.6701,1990,31.1396,30.90485,0.018539,million metric tonnes,Natural carbon capture is computed as the rati...
4,Algeria,Forest land,1990,1667.0,Carbon stock in living biomass,30.5136,1990,31.1396,30.82660,0.018492,million metric tonnes,Natural carbon capture is computed as the rati...
...,...,...,...,...,...,...,...,...,...,...,...,...
527404,Zimbabwe,Planted Forest,2021,108.0,Carbon stock in living biomass,1004.8564,1990,1189.2814,1097.06890,10.158045,million metric tonnes,Natural carbon capture is computed as the rati...
527405,Zimbabwe,Planted Forest,2021,108.0,Carbon stock in living biomass,1002.2235,1990,1189.2814,1095.75245,10.145856,million metric tonnes,Natural carbon capture is computed as the rati...
527406,Zimbabwe,Planted Forest,2021,108.0,Carbon stock in living biomass,999.5906,1990,1189.2814,1094.43600,10.133667,million metric tonnes,Natural carbon capture is computed as the rati...
527407,Zimbabwe,Planted Forest,2021,108.0,Carbon stock in living biomass,996.7833,1990,1189.2814,1093.03235,10.120670,million metric tonnes,Natural carbon capture is computed as the rati...


In [18]:
clean_carbon_capture.to_csv("cleanedCARBON.csv")

## Nitrogen ##

In [19]:
pip install xlrd

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [21]:
epi_file = '2018-epi.xlsx' # data is from EPI's biannual report (for this, it's 2018)
epi = pd.read_excel(epi_file, '2018EPI_ScoresCurrent')
epi.head()

,code,iso,country,EPI.current,HLT.current,AIR.current,H2O.current,HMT.current,ECO.current,BDH.current,...,MTR.current,DCT.current,DPT.current,DMT.current,DNT.current,DBT.current,DST.current,DXT.current,WWT.current,SNM.current
0,4,AFG,Afghanistan,37.74,36.76,44.67,25.75,0.00,38.40,13.44,...,-9999.00,23.47,-9999.00,53.62,50.10,67.66,82.88,100.00,0.00,44.77
1,8,ALB,Albania,65.46,65.67,65.47,66.56,62.89,65.32,75.37,...,61.77,53.77,100.00,73.30,63.65,72.61,83.25,88.90,80.73,22.61
2,12,DZA,Algeria,57.18,80.95,93.98,60.26,35.60,41.34,49.33,...,60.58,17.14,46.35,37.76,23.99,6.93,83.63,8.68,91.60,34.26
3,24,AGO,Angola,37.44,33.79,44.66,9.20,40.08,39.88,39.19,...,38.35,55.00,40.18,36.11,63.78,58.16,59.85,55.99,0.00,22.70
4,28,ATG,Antigua and Barbuda,59.18,76.53,89.13,53.33,52.04,47.61,88.49,...,59.13,0.00,-9999.00,22.94,85.88,47.50,46.86,47.03,64.44,7.26


In [22]:
epi[['iso', 'country','SNM.current']].head() #snm is on nitrogen management

,iso,country,SNM.current
0,AFG,Afghanistan,44.77
1,ALB,Albania,22.61
2,DZA,Algeria,34.26
3,AGO,Angola,22.70
4,ATG,Antigua and Barbuda,7.26


In [23]:
#missing data:
epi1=epi[['iso', 'country','SNM.current']]
epi1[epi1.isna().any(axis=1)]
#apparently there is no country in the list missing Nitrogen data!

,iso,country,SNM.current


#### WITH SSPI' BULK DATA

In [24]:
snm_2022=pd.read_csv('SNM_raw (1).csv')
snm_2022.head(15) #a lot of strange values but I think goal-posting will fix the problem

,code,iso,country,SNM.raw.1961,SNM.raw.1962,SNM.raw.1963,SNM.raw.1964,SNM.raw.1965,SNM.raw.1966,SNM.raw.1967,...,SNM.raw.2006,SNM.raw.2007,SNM.raw.2008,SNM.raw.2009,SNM.raw.2010,SNM.raw.2011,SNM.raw.2012,SNM.raw.2013,SNM.raw.2014,SNM.raw.2015
0,4,AFG,Afghanistan,0.000000,0.790609,0.802655,0.785905,0.782866,0.801205,0.762432,...,0.723017,0.655114,0.640026,0.608648,0.611114,0.737539,0.823029,0.686732,0.648274,0.668763
1,8,ALB,Albania,1.101448,1.051832,1.051140,1.014870,1.036413,0.963800,0.971312,...,0.956283,0.953287,0.891333,0.877849,0.835016,0.831228,0.834036,0.810873,0.812218,0.851394
2,12,DZA,Algeria,0.925632,1.138094,1.024589,0.901072,0.963538,0.935814,0.895745,...,0.745333,0.773993,0.756695,0.759777,0.726148,0.697795,0.694354,0.686745,0.778200,0.752511
3,20,AND,Andorra,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
4,24,AGO,Angola,0.880651,0.887588,0.893791,0.883528,0.886632,0.898583,0.907756,...,0.926125,0.947929,0.910338,0.924042,0.935621,0.902627,0.973230,0.891369,0.953422,0.963949
5,660,AIA,Anguilla,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000
6,28,ATG,Antigua and Barbuda,1.137189,1.107661,1.040947,1.026203,1.162898,1.208191,1.224500,...,1.326673,1.313726,1.315688,1.317395,1.302731,1.314166,1.314998,1.313858,1.309752,1.295053
7,32,ARG,Argentina,0.800651,0.794512,0.863438,0.924786,0.756822,0.776667,0.809538,...,0.357478,0.360974,0.314453,0.322126,0.351357,0.327483,0.299505,0.322863,0.311469,0.295102
8,51,ARM,Armenia,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,...,0.695424,0.516132,0.533623,0.640364,0.686859,0.551368,0.633244,0.593491,0.505360,0.578095
9,533,ABW,Aruba,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,...,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000,-8888.000000


In [25]:
a = np.array(snm_2022[snm_2022.columns[-55:]].values.tolist())
snm_2022_v2=snm_2022[snm_2022.columns[-55:]] = np.where(a <-100, 0, a).tolist()

In [26]:
snm_2022[snm_2022.columns[-55:]] = snm_2022[snm_2022.columns[-55:]].apply(lambda x: [y if y > -1000 else 0 for y in x])
snm_2022.head()

,code,iso,country,SNM.raw.1961,SNM.raw.1962,SNM.raw.1963,SNM.raw.1964,SNM.raw.1965,SNM.raw.1966,SNM.raw.1967,...,SNM.raw.2006,SNM.raw.2007,SNM.raw.2008,SNM.raw.2009,SNM.raw.2010,SNM.raw.2011,SNM.raw.2012,SNM.raw.2013,SNM.raw.2014,SNM.raw.2015
0,4,AFG,Afghanistan,0.000000,0.790609,0.802655,0.785905,0.782866,0.801205,0.762432,...,0.723017,0.655114,0.640026,0.608648,0.611114,0.737539,0.823029,0.686732,0.648274,0.668763
1,8,ALB,Albania,1.101448,1.051832,1.051140,1.014870,1.036413,0.963800,0.971312,...,0.956283,0.953287,0.891333,0.877849,0.835016,0.831228,0.834036,0.810873,0.812218,0.851394
2,12,DZA,Algeria,0.925632,1.138094,1.024589,0.901072,0.963538,0.935814,0.895745,...,0.745333,0.773993,0.756695,0.759777,0.726148,0.697795,0.694354,0.686745,0.778200,0.752511
3,20,AND,Andorra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,24,AGO,Angola,0.880651,0.887588,0.893791,0.883528,0.886632,0.898583,0.907756,...,0.926125,0.947929,0.910338,0.924042,0.935621,0.902627,0.973230,0.891369,0.953422,0.963949


In [27]:
snm_2022['Description']= 'Sustainable Nitrogen Management is a unitless index, ranging from 0 to 2, geometrically calculating the country’s current position relative to its reference point on two attributes: nitrogen efficiency in agriculture and crop yield in relation to environmental performance' 

In [28]:
snm_2022.head()

,code,iso,country,SNM.raw.1961,SNM.raw.1962,SNM.raw.1963,SNM.raw.1964,SNM.raw.1965,SNM.raw.1966,SNM.raw.1967,...,SNM.raw.2007,SNM.raw.2008,SNM.raw.2009,SNM.raw.2010,SNM.raw.2011,SNM.raw.2012,SNM.raw.2013,SNM.raw.2014,SNM.raw.2015,Description
0,4,AFG,Afghanistan,0.000000,0.790609,0.802655,0.785905,0.782866,0.801205,0.762432,...,0.655114,0.640026,0.608648,0.611114,0.737539,0.823029,0.686732,0.648274,0.668763,Sustainable Nitrogen Management is a unitless ...
1,8,ALB,Albania,1.101448,1.051832,1.051140,1.014870,1.036413,0.963800,0.971312,...,0.953287,0.891333,0.877849,0.835016,0.831228,0.834036,0.810873,0.812218,0.851394,Sustainable Nitrogen Management is a unitless ...
2,12,DZA,Algeria,0.925632,1.138094,1.024589,0.901072,0.963538,0.935814,0.895745,...,0.773993,0.756695,0.759777,0.726148,0.697795,0.694354,0.686745,0.778200,0.752511,Sustainable Nitrogen Management is a unitless ...
3,20,AND,Andorra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Sustainable Nitrogen Management is a unitless ...
4,24,AGO,Angola,0.880651,0.887588,0.893791,0.883528,0.886632,0.898583,0.907756,...,0.947929,0.910338,0.924042,0.935621,0.902627,0.973230,0.891369,0.953422,0.963949,Sustainable Nitrogen Management is a unitless ...


In [29]:
snm_2022.to_csv("cleanedSNM_2022.csv")

## Beef Market

### 1. beef and buffalo production

In [54]:
beef_production=pd.read_csv('beef-and-buffalo-meat-production-tonnes.csv')
beef_production

,Entity,Code,Year,"Meat, beef and buffalo | 00001806 || Production | 005510 || tonnes"
0,Afghanistan,AFG,1961,43000.00
1,Afghanistan,AFG,1962,45800.00
2,Afghanistan,AFG,1963,47250.00
3,Afghanistan,AFG,1964,48000.00
4,Afghanistan,AFG,1965,48700.00
...,...,...,...,...
13805,Zimbabwe,ZWE,2017,609788.00
13806,Zimbabwe,ZWE,2018,616480.00
13807,Zimbabwe,ZWE,2019,629189.00
13808,Zimbabwe,ZWE,2020,628746.00


In [62]:
#missing data:
bp=beef_production[['Meat, beef and buffalo | 00001806 || Production | 005510 || tonnes']]
bp[bp.isna().any(axis=1)]
#also no missing countries...

,"Meat, beef and buffalo | 00001806 || Production | 005510 || tonnes"


In [56]:
bp.max()

Meat, beef and buffalo | 00001806 || Production | 005510 || tonnes    76768270.0
dtype: float64

### 2. per capita meat consumption 

In [57]:
beef_consumption=pd.read_csv('per-capita-meat-consumption-by-type-kilograms-per-year.csv')
beef_consumption

,Entity,Code,Year,"Meat, Other | 00002735 || Food available for consumption | 0645pc || kilograms per year per capita","Meat, sheep and goat | 00002732 || Food available for consumption | 0645pc || kilograms per year per capita","Meat, beef | 00002731 || Food available for consumption | 0645pc || kilograms per year per capita","Meat, pig | 00002733 || Food available for consumption | 0645pc || kilograms per year per capita","Meat, poultry | 00002734 || Food available for consumption | 0645pc || kilograms per year per capita"
0,Afghanistan,AFG,1961,0.865845,8.332486,4.889479,NaN,0.641744
1,Afghanistan,AFG,1962,0.896969,8.072717,5.106605,NaN,0.672726
2,Afghanistan,AFG,1963,1.091328,8.251259,5.160862,NaN,0.673156
3,Afghanistan,AFG,1964,1.030673,8.520909,5.132955,NaN,0.683714
4,Afghanistan,AFG,1965,1.082147,8.820517,5.094256,NaN,0.714625
...,...,...,...,...,...,...,...,...
12355,Zimbabwe,ZWE,2016,2.601678,1.436748,43.684890,1.572656,5.174232
12356,Zimbabwe,ZWE,2017,2.499662,1.804775,41.403670,0.743143,5.086185
12357,Zimbabwe,ZWE,2018,2.388530,1.822573,40.988712,0.517995,4.383768
12358,Zimbabwe,ZWE,2019,2.375002,1.774098,40.975937,0.686748,4.473397


In [58]:
#not sure which column of data we're taking...
bc=beef_consumption[['Entity','Meat, beef | 00002731 || Food available for consumption | 0645pc || kilograms per year per capita']]
bc[bc.isna().any(axis=1)]
#no missing data!

,Entity,"Meat, beef | 00002731 || Food available for consumption | 0645pc || kilograms per year per capita"


In [76]:
methane=pd.merge(beef_production, beef_consumption, on='Entity').rename(columns={'Entity':'Country', 'Code_x':'Country Code', 'Year_x':'Meat Production Year', 'Meat, beef and buffalo | 00001806 || Production | 005510 || tonnes': 'Meat (Beef & Buffalo) Production (tonnes)','Code_y': 'Meat Consumption Country Code', 'Year_y':'Meat Consumption Year', 'Meat, beef | 00002731 || Food available for consumption | 0645pc || kilograms per year per capita':'Per Capital Meat (Beef & Buffalo) Consumption (kg per year)'})
methane.drop(columns=['Meat, Other | 00002735 || Food available for consumption | 0645pc || kilograms per year per capita','Meat, sheep and goat | 00002732 || Food available for consumption | 0645pc || kilograms per year per capita', 'Meat, pig | 00002733 || Food available for consumption | 0645pc || kilograms per year per capita', 'Meat, poultry | 00002734 || Food available for consumption | 0645pc || kilograms per year per capita'])

,Country,Country Code,Meat Production Year,Meat (Beef & Buffalo) Production (tonnes),Meat Consumption Country Code,Meat Consumption Year,Per Capital Meat (Beef & Buffalo) Consumption (kg per year)
0,Afghanistan,AFG,1961,43000.00,AFG,1961,4.889479
1,Afghanistan,AFG,1961,43000.00,AFG,1962,5.106605
2,Afghanistan,AFG,1961,43000.00,AFG,1963,5.160862
3,Afghanistan,AFG,1961,43000.00,AFG,1964,5.132955
4,Afghanistan,AFG,1961,43000.00,AFG,1965,5.094256
...,...,...,...,...,...,...,...
710572,Zimbabwe,ZWE,2021,806293.75,ZWE,2016,43.684890
710573,Zimbabwe,ZWE,2021,806293.75,ZWE,2017,41.403670
710574,Zimbabwe,ZWE,2021,806293.75,ZWE,2018,40.988712
710575,Zimbabwe,ZWE,2021,806293.75,ZWE,2019,40.975937


In [71]:
methane.to_csv("cleanedMethane.csv")

### Municipal Solid Waste Generation

In [35]:
waste=pd.read_csv('country_level_data.csv')

In [36]:
waste.drop(columns=['region_id']).head()

,iso3c,country_name,income_id,gdp,composition_food_organic_waste_percent,composition_glass_percent,composition_metal_percent,composition_other_percent,composition_paper_cardboard_percent,composition_plastic_percent,...,waste_treatment_controlled_landfill_percent,waste_treatment_incineration_percent,waste_treatment_landfill_unspecified_percent,waste_treatment_open_dump_percent,waste_treatment_other_percent,waste_treatment_recycling_percent,waste_treatment_sanitary_landfill_landfill_gas_system_percent,waste_treatment_unaccounted_for_percent,waste_treatment_waterways_marine_percent,where_where_is_this_data_measured
0,ABW,Aruba,HIC,35563.312500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,11.0,NaN,89.0,NaN,NaN
1,AFG,Afghanistan,LIC,2057.062256,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other
2,AGO,Angola,LMC,8036.690430,51.8,6.7,4.4,11.50,11.9,13.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALB,Albania,UMC,13724.058594,51.4,4.5,4.8,15.21,9.9,9.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Some disposal sites
4,AND,Andorra,HIC,43711.800781,31.2,8.2,2.6,11.60,35.1,11.3,...,NaN,52.1,NaN,NaN,NaN,NaN,NaN,47.9,NaN,NaN


In [103]:
waste[['iso3c', 'country_name', 'total_msw_total_msw_generated_tons_year']].head()

,country_name,total_msw_total_msw_generated_tons_year
0,Aruba,8.813202e+04
1,Afghanistan,5.628525e+06
2,Angola,4.213644e+06
3,Albania,1.087447e+06
4,Andorra,4.300000e+04


In [104]:
muni_waste=waste[['iso3c', 'country_name', 'total_msw_total_msw_generated_tons_year']]
muni_waste[muni_waste.isna().any(axis=1)]
#the following are missing

,iso3c,country_name,total_msw_total_msw_generated_tons_year
183,SXM,Sint Maarten (Dutch part),NaN
186,TCA,Turks and Caicos Islands,NaN
198,TWN,NaN,7336000.0


### E-waste

In [39]:
waste[['iso3c', 'country_name', 'special_waste_e_waste_tons_year']].head()

,iso3c,country_name,special_waste_e_waste_tons_year
0,ABW,Aruba,NaN
1,AFG,Afghanistan,20000.0
2,AGO,Angola,92000.0
3,ALB,Albania,20000.0
4,AND,Andorra,NaN


In [108]:
#missing data - lots of them!
ewaste=waste[['country_name', 'special_waste_e_waste_tons_year']]
c=ewaste[ewaste.isna().any(axis=1)]

### Waste Collection Rate

In [41]:
waste[['iso3c', 'country_name', 'waste_collection_coverage_total_percent_of_population']].head()

,iso3c,country_name,waste_collection_coverage_total_percent_of_population
0,ABW,Aruba,NaN
1,AFG,Afghanistan,NaN
2,AGO,Angola,23.1
3,ALB,Albania,60.0
4,AND,Andorra,100.0


In [105]:
#a lot of missing data
colrate=waste[['country_name', 'waste_collection_coverage_total_percent_of_population']]
a=colrate[colrate.isna().any(axis=1)]
a

,country_name,waste_collection_coverage_total_percent_of_population
0,Aruba,NaN
1,Afghanistan,NaN
5,United Arab Emirates,NaN
7,Armenia,NaN
10,Australia,NaN
11,Austria,NaN
13,Burundi,NaN
14,Belgium,NaN
15,Benin,NaN
17,Bangladesh,NaN


### Recycling Rate

In [43]:
waste[['iso3c', 'country_name', 'waste_treatment_recycling_percent']].head()

,iso3c,country_name,waste_treatment_recycling_percent
0,ABW,Aruba,11.0
1,AFG,Afghanistan,NaN
2,AGO,Angola,NaN
3,ALB,Albania,NaN
4,AND,Andorra,NaN


In [106]:
#also a lot of missing data
rerate=waste[['country_name', 'waste_treatment_recycling_percent']]
b=rerate[rerate.isna().any(axis=1)]

### COUNTRIES WITH MISSING WASTE DATA

In [45]:
combined_df=[a,b,c]
missing_waste_data = pd.concat(combined_df)
missing_waste_data

,iso3c,country_name,waste_collection_coverage_total_percent_of_population,waste_treatment_recycling_percent,special_waste_e_waste_tons_year
0,ABW,Aruba,NaN,NaN,NaN
1,AFG,Afghanistan,NaN,NaN,NaN
5,ARE,United Arab Emirates,NaN,NaN,NaN
7,ARM,Armenia,NaN,NaN,NaN
10,AUS,Australia,NaN,NaN,NaN
11,AUT,Austria,NaN,NaN,NaN
13,BDI,Burundi,NaN,NaN,NaN
14,BEL,Belgium,NaN,NaN,NaN
15,BEN,Benin,NaN,NaN,NaN
17,BGD,Bangladesh,NaN,NaN,NaN


In [72]:
missing_waste_data.to_csv("missing_waste_data.csv")

In [111]:
#tot_waste=[muni_waste,ewaste,colrate,rerate]
waste1=pd.merge(muni_waste, ewaste, on='country_name')
waste2=pd.merge(colrate,rerate, on='country_name')
total_waste=pd.merge(waste1, waste2, on='country_name')
total_waste.rename(columns={'iso3c':'iso3 (Country Code)','country_name':'Country Name','special_waste_e_waste_tons_year':'E Waste (tonnes/year)','total_msw_total_msw_generated_tons_year':'Municipal Solid Waste (tonnes/year)','waste_collection_coverage_total_percent_of_population':'Waste Collection Coverage Rate','waste_treatment_recycling_percent':'Recycling Rate'})

,iso3 (Country Code),Country Name,Municipal Solid Waste (tonnes/year),E Waste (tonnes/year),Waste Collection Coverage Rate,Recycling Rate
0,ABW,Aruba,8.813202e+04,NaN,NaN,11.000000
1,AFG,Afghanistan,5.628525e+06,20000.00,NaN,NaN
2,AGO,Angola,4.213644e+06,92000.00,23.10,NaN
3,ALB,Albania,1.087447e+06,20000.00,60.00,NaN
4,AND,Andorra,4.300000e+04,NaN,100.00,NaN
5,ARE,United Arab Emirates,5.617682e+06,134000.00,NaN,20.000000
6,ARG,Argentina,1.791055e+07,291700.00,89.91,6.000000
7,ARM,Armenia,4.928000e+05,14000.00,NaN,NaN
8,ASM,American Samoa,1.898949e+04,NaN,100.00,NaN
9,ATG,Antigua and Barbuda,3.058500e+04,1100.00,98.61,NaN


In [112]:
total_waste.to_csv("cleanedWasteData.csv")

## Child labor

### 1. child labor rate (if available) (UNSDG 8.7.1)

In [46]:
cl_file = 'data.xlsx' 
cl = pd.read_excel(cl_file, 'Goal8') #name of the specific sheet within the spreadsheet
cl.head()

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,LowerBound,BasePeriod,Source,GeoInfoUrl,FootNote,Age,Nature,Reporting Type,Sex,Units
0,8,8.7,8.7.1,SL_TLF_CHLDEA,Proportion of children engaged in economic act...,4,Afghanistan,2020,12.6,2020,...,NaN,NaN,"IELFS 2020, UNICEF and ILO calculations",NaN,These estimates differ from published national...,5-17,CA,G,MALE,PERCENT
1,8,8.7,8.7.1,SL_TLF_CHLDEA,Proportion of children engaged in economic act...,4,Afghanistan,2020,5.1,2020,...,NaN,NaN,"IELFS 2020, UNICEF and ILO calculations",NaN,These estimates differ from published national...,5-17,CA,G,FEMALE,PERCENT
2,8,8.7,8.7.1,SL_TLF_CHLDEA,Proportion of children engaged in economic act...,4,Afghanistan,2020,9.0,2020,...,NaN,NaN,"IELFS 2020, UNICEF and ILO calculations",NaN,These estimates differ from published national...,5-17,CA,G,BOTHSEX,PERCENT
3,8,8.7,8.7.1,SL_TLF_CHLDEA,Proportion of children engaged in economic act...,8,Albania,2010,3.4,2010,...,NaN,NaN,"CLS 2010, ILO Harmonized Microdata, https://il...",NaN,,5-17,CA,G,MALE,PERCENT
4,8,8.7,8.7.1,SL_TLF_CHLDEA,Proportion of children engaged in economic act...,8,Albania,2010,2.1,2010,...,NaN,NaN,"CLS 2010, ILO Harmonized Microdata, https://il...",NaN,,5-17,CA,G,FEMALE,PERCENT


In [47]:
cl = cl.drop(columns=['Goal','Indicator', 'SeriesCode', 'SeriesDescription','GeoAreaCode','Time_Detail','TimeCoverage','UpperBound','LowerBound','BasePeriod','GeoInfoUrl','FootNote','Nature','Reporting Type'])
cl1= cl[cl["Sex"] == "BOTHSEX"]
cl1.head()

,Target,GeoAreaName,TimePeriod,Value,Source,Age,Sex,Units
2,8.7,Afghanistan,2020,9.000,"IELFS 2020, UNICEF and ILO calculations",5-17,BOTHSEX,PERCENT
5,8.7,Albania,2010,2.800,"CLS 2010, ILO Harmonized Microdata, https://il...",5-17,BOTHSEX,PERCENT
6,8.7,Algeria,2019,1.900,"MICS 2018-19, UNICEF and ILO calculations",5-17,BOTHSEX,PERCENT
9,8.7,Angola,2016,9.666,"DHS 2015-16, UNICEF and ILO calculations",5-17,BOTHSEX,PERCENT
12,8.7,Bangladesh,2019,5.900,"MICS 2019, UNICEF and ILO calculations",5-17,BOTHSEX,PERCENT


In [48]:
grouped_by_age_ranges = cl1.groupby("GeoAreaName")["Age"].unique().to_frame()
grouped_by_age_ranges

,Age
GeoAreaName,
Afghanistan,[5-17]
Albania,"[5-17, 5-14]"
Algeria,[5-17]
Angola,[5-17]
Armenia,[5-17]
Bangladesh,[5-17]
Barbados,[5-14]
Belarus,[5-17]
Belize,[5-17]


In [4]:
import os

os.system('jupyter nbconvert --to html non_api_data.ipynb')

[NbConvertApp] Converting notebook non_api_data.ipynb to html
[NbConvertApp] Writing 485935 bytes to non_api_data.html


0